In [ ]:
# https://leemeng.tw/practical-pandas-tutorial-for-aspiring-data-scientists.html

In [1]:
import pandas as pd
pd.__version__

'0.24.0'

In [2]:
dic = {
    "col 1": [1, 2, 3], 
    "col 2": [10, 20, 30],
    "col 3": list('xyz'),
    "col 4": ['a', 'b', 'c'],
    "col 5": pd.Series(range(3))
}
df = pd.DataFrame(dic)
df

,col 1,col 2,col 3,col 4,col 5
0,1,10,x,a,0
1,2,20,y,b,1
2,3,30,z,c,2


In [3]:
rename_dic = {"col 1": "x", "col 2": "10x"}
df.rename(rename_dic, axis=1)

,x,10x,col 3,col 4,col 5
0,1,10,x,a,0
1,2,20,y,b,1
2,3,30,z,c,2


In [4]:
df.columns = ['x(new)', '10x(new)'] + list(df.columns[2:])
df

,x(new),10x(new),col 3,col 4,col 5
0,1,10,x,a,0
1,2,20,y,b,1
2,3,30,z,c,2


In [5]:
pd.util.testing.makeDataFrame().head(10)

,A,B,C,D
sSC4UdSsUm,-1.030154,-0.190393,-0.827299,-0.209791
McXqHQBWXs,0.461431,0.685511,-0.605518,0.448542
hM3qpHe5QL,-1.122784,-0.055938,-0.405803,-2.185044
kgTWMhY6fU,1.041656,-0.261223,0.594737,-0.195325
hOw61juHl4,0.633912,0.303646,-2.398433,0.515381
hAJrrw1c5F,-0.923928,0.621389,1.231499,0.935920
9QctiD3APb,-1.782391,1.736956,1.867148,-1.288650
jeEdlHe9vi,1.245576,-1.804953,-0.023825,-0.789668
PO6RogO0ek,-2.174276,0.167186,0.410782,0.370436
yA4r0Q0MZE,0.294434,-0.308871,0.211227,-0.200681


In [6]:
df = pd.read_csv('http://bit.ly/kaggletrain')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# 你可以用類似的方式爬取任何網路上的公開數據集
base_url = "https://data.taipei/api/getDatasetInfo/downloadResource?id={}&rid={}"
_id = "2f238b4f-1b27-4085-93e9-d684ef0e2735"
rid = "ea731a84-e4a1-4523-b981-b733beddbc1f"
csv_url = base_url.format(_id, rid)
df_raw = pd.read_csv(csv_url, encoding='big5')

# 複製一份做處理
df = df_raw.copy()

# 計算不同區不同性別的死亡、受傷人數
df['區序'] = df['區序'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df = (df[df['性別'].isin([1, 2])]
      .groupby(['區序', '性別'])[['死亡人數', '受傷人數']]
      .sum()
      .reset_index()
      .sort_values('受傷人數'))

df['性別'] = df['性別'].apply(lambda x: '男性' if x == 1 else '女性')
df = df.reset_index().drop('index', axis=1)

# 顯示結果
display(df_raw.head())
display(df.head())

,發生年,發生月,發生日,發生時,發生分,處理別,區序,肇事地點,死亡人數,受傷人數,當事人序,車種,性別,年齡,受傷程度,4天候,7速限,8道路型態,9事故位置
0,107,3,29,15,54,2,01大同區,大同區民權西路108號,0,1,1,B03,2,41.0,3.0,8,50,14.0,9.0
1,107,3,29,15,54,2,01大同區,大同區民權西路108號,0,1,2,C03,2,58.0,2.0,8,50,14.0,9.0
2,107,1,7,17,42,2,01大同區,大同區重慶北路2段與南京西路口,0,1,1,B01,1,59.0,3.0,6,40,4.0,2.0
3,107,1,7,17,42,2,01大同區,大同區重慶北路3段與南京西路口,0,1,2,C03,1,18.0,2.0,6,40,4.0,2.0
4,107,1,14,9,56,2,01大同區,大同區承德路3段與民族西路口,0,1,1,C03,1,20.0,2.0,8,50,4.0,1.0


,區序,性別,死亡人數,受傷人數
0,南港區,女性,2,799
1,萬華區,女性,2,1084
2,松山區,女性,30,1146
3,大同區,女性,3,1336
4,士林區,女性,3,1660


In [17]:
import chartify
import time
from selenium import webdriver

driver = webdriver.Chrome('../../chromedriver')

ch = chartify.Chart(
    x_axis_type='categorical', 
    y_axis_type='categorical')

ch.plot.heatmap(
    data_frame=df, 
    y_column='性別', 
    x_column='區序', 
    color_column='受傷人數', 
    text_column='受傷人數', 
    color_palette='Reds', 
    text_format='{:,.0f}')

(ch.set_title('2017 年台北市各行政區交通意外受傷人數')
 .set_subtitle('性別分計')
 .set_source_label("Data.Taipei")
 .axes.set_xaxis_label('性別')
 .axes.set_yaxis_label('行政區')
 .show('png'))

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 80


In [16]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service('../../chromedriver')
service.start()
driver = webdriver.Remote(service.service_url)
driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
driver.quit()

WebDriverException: Message: Desired Capabilities can't be None


In [22]:
# https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/583492/
# -*- coding: utf-8 -*-
"""
使用NumPy快速讀取CSV檔案。
"""
import numpy as np
# 採用字串陣列讀取檔案
tmp = np.loadtxt("hello.csv", dtype=np.str, delimiter=",")
# 將部分陣列的值進行轉換
data = tmp[1:,1:].astype(np.float)
print (data)
# 定義結構陣列元素的型別
persontype = np.dtype({
'names':['name', 'age', 'weight', 'height'],
'formats':['S32','i', 'f', 'f']})
f = open("hello.csv")
f.readline() # 跳過第一行
data = np.loadtxt(f, dtype=persontype, delimiter=",")
f.close()
print (data)

[[ 30.  75. 165.]
 [ 45.  60. 170.]
 [ 15.  30. 120.]]


UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-1: ordinal not in range(256)

In [25]:
# https://blog.csdn.net/zhubao124/article/details/80720694
from numpy import *
 
a = arange(20).reshape(4,5)
print(a)
 
#delimiter:分隔符属性，指定以什么字符分隔；对一维数组不起作用
savetxt('a.csv',a,fmt='%d',delimiter=',')#以逗号，分隔
 
#usecols：指定获取对应的列数的数据
b = loadtxt('a.csv',dtype=int,delimiter=',',usecols=(1,3,4))
print(b)
 
#unpack：将usecols属性指定获取对应的列数的数据，分别存入到左侧的变量中
x,y,z = loadtxt('a.csv',dtype=int,delimiter=',',usecols=(1,3,4),unpack=True)
print(x)
print(y)
print(z)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
[[ 1  3  4]
 [ 6  8  9]
 [11 13 14]
 [16 18 19]]
[ 1  6 11 16]
[ 3  8 13 18]
[ 4  9 14 19]


In [29]:
# https://blog.csdn.net/zhubao124/article/details/80720694

import numpy as np
a=np.arange(15).reshape(5,3)
np.save('a.npy',a)
b=np.load('a.npy')
print(b)

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]


In [4]:
from tkinter import *
tk=Tk()
var=IntVar()
#标签控件，显示文本和位图，展示在第一行
Label(tk,text="First").grid(row=0,sticky=E)#靠右
Label(tk,text="Second").grid(row=1,sticky=W)#第二行，靠左
#输入控件
Entry(tk).grid(row=0,column=1,padx=10,pady=10)
Entry(tk).grid(row=1,column=1)
#多选框插件
button=Checkbutton(tk,text="Precerve aspect",variable=var)
button.grid(row=2,columnspan=2,sticky=W)
#插入图片
photo=PhotoImage(file="IMG_002.png")
label=Label(image=photo)
label.grid(row=0,column=2,rowspan=2,columnspan=2,
           sticky=W+E+N+S, padx=5, pady=5)#合并两行，两列，居中，四周外延5个长度
#按钮控件
button1=Button(tk,text="Zoom in")
button1.grid(row=2,column=2)
button2=Button(tk,text="Zoom out")
button2.grid(row=2,column=3)
#主事件循环
mainloop()

In [ ]:
寫拖拉方塊
https://www.delftstack.com/zh-tw/tutorial/tkinter-tutorial/tkinter-combobox/
    https://www.tianqiweiqi.com/python-tkinter-place-pack-grid.html